# Galactic Chemical Evolution Activity

*Original author: B. Côté; modified by B. Világos and V. Hegedűs*

In this notebook, you will use OMEGA+
([Côté et al. 2018](http://adsabs.harvard.edu/abs/2018ApJ...859...67C)), a simple galactic chemical evolution code designed to connect nuclear astrophysics with astronomical observations.

**Open-source Python packages**
* [NuPyCEE](https://github.com/NuGrid/NuPyCEE) - NuGrid Python Chemical Evolution Environment
* [JINAPyCEE](https://github.com/becot85/JINAPyCEE) - JINA Python Chemical Evolution Environment

In [ ]:
# This will install OMEGA and OMEGA+.
# Only run for the first time! If you accidentally run it more than once,
# you will get an error, because OMEGA is already installed.
# Ignore the error, it won't affect your installation.
# If you use Google Colab, after some time of inactivity you may have to install it again.

!git clone https://github.com/NuGrid/NuPyCEE.git
!git clone https://github.com/becot85/JINAPyCEE.git

In [ ]:
# Import the OMEGA+ code and standard packages
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from JINAPyCEE import omega_plus
from NuPyCEE import stellab
%matplotlib inline

# 1. Star Formation and Gas Reservoirs

Here you will learn about the interplay between galactic inflows, star formation, and galactic outflows. The model consists of a galaxy surrounded by a large gas reservoir called the circumgalactic medium. The galaxy is composed of gas and stars. The model follows galactic inflows, representing the transfer of gas from the circumgalactic medium into the galaxy, as well as galactic outflows, representing the transfer of gas from the galaxy into the circumgalactic medium.

The simulation will start with $1.56\times10^{11}$ M$_\odot$ of gas inside the circumgalactic medium. Once gas will be introduced inside the galaxy, stars will start to form.  The equation describing the rate of change of the mass of gas **inside** the galaxy is

$$\dot{M}_\mathrm{gas,in} = \dot{M}_\mathrm{inflow} - \dot{M}_\star + \dot{M}_\mathrm{ej} - \dot{M}_\mathrm{outflow} \mathrm{\quad[M_\odot/yr]}$$

* $M_\mathrm{gas,in}$: Mass of gas inside the galaxy.
* $\dot{M}_\mathrm{inflow}$: Galactic inflow rate.
* $\dot{M}_\star$: Star formation rate. Conversion of gas into stars.
* $\dot{M}_\mathrm{ej}$: Stellar ejecta. Mass ejected by stars when they die (ex. supernova explosion).
* $\dot{M}_\mathrm{outflow}$: Galactic outflow rate.

### Run a basic model
Run the cell below to launch the model. **For now, simply run the model without changing the parameters**.

In [ ]:
# Star formation efficiency.
# This is controling how fast the gas inside the galaxy is converted into stars.
# You can vary this parameter between 0.001 and 0.05
# (you can use values outside this range, but that is unphysical)
sfe = 0.001

# Galactic outflow rate.
# You can vary this parameter between 0.0 and 5.0
mass_loading = 0.0

# !!Do not modify the following line!!
# Run OMEGA+ with the first set of parameters
o = omega_plus.omega_plus(sfe=sfe, mass_loading=mass_loading, special_timesteps=60, \
                          table='yield_tables/agb_and_massive_stars_K10_K06_0.5HNe.txt', \
                          t_inflow=5e9, t_star=1.0e8, f_halo_to_gal_out=0.0)

### Extract the Information

The equation is solved in timesteps, step by step. You usually want to extract values from the arrays containing the properties of the galaxy that change in time (e.g. the mass of an isotope). Use the array you are interested in and give the index of the timestep that you want to have.

You can extract the age of the galaxy for a specific timestep by calling the array **o.inner.history.age**. An example:





In [ ]:
# The equations describing the evolution of the galaxy are solved step by step.

# Age of the galaxy at each timestep [Gyr, billion of years].
t_sim = o.inner.history.age[:-1]

# Print the number of timesteps.
print(o.inner.nb_timesteps)

60


The 'o' object that you ran in a cell above contains all the values of the OMEGA+ run. OMEGA+ uses OMEGA internally to run the evolution of the galaxy itself, then it treats the interaction with the outer gas reservoir.

Generally, if you want to extract a number that is a property of the galaxy itself, you will use the values provided by OMEGA. You can access the values stored by OMEGA by calling **o.inner**. If you want to acces something related to the circumgalactic medium, you should just call **o**, because that is a value handled by OMEGA+ and the inner OMEGA doesn't know about that.

Note that the arrays that contain some masses are usually 2D arrays, where the first index (a row) is the timestep, and the second index (a column) is corresponding to each isotope. If you want to treat the total mass, you should sum for all isotopes (sum the array on axis 1).

**Here are some important arrays**:
* Mass of isotopes in the gas inside the circumgalactic medium, for each timestep
    * **o.ymgal_outer**[ timestep index ][ isotope index ]
* Mass of isotopes in the gas inside the galaxy, for each timestep
    * **o.inner.ymgal**[ timestep index ][ isotope index ]
* Mass of gas converted into stars at each timestep (*this is not the total cumulated mass of stars*)
    * **o.inner.history.m_locked**[ timestep index ]
* Mass of isotopes ejected by stars at each timestep
    * **o.inner.mdot**[ timestep index ][ isotope index ]

In [ ]:
# The mass of the two gas reservoirs is decomposed into isotopes.
# To calculate the total mass, at each timestep, you must sum the mass of all isotopes

# Print the number of isotopes.
print('Number of isotopes =',o.inner.nb_isotopes)

# Print the list of isotopes
print('List of isotopes =',o.inner.history.isotopes)

Number of isotopes = 283
List of isotopes = ['H-1', 'H-2', 'He-3', 'He-4', 'Li-6', 'Li-7', 'Be-9', 'B-10', 'B-11', 'C-12', 'C-13', 'N-14', 'N-15', 'O-16', 'O-17', 'O-18', 'F-19', 'Ne-20', 'Ne-21', 'Ne-22', 'Na-23', 'Mg-24', 'Mg-25', 'Mg-26', 'Al-27', 'Si-28', 'Si-29', 'Si-30', 'P-31', 'S-32', 'S-33', 'S-34', 'S-36', 'Cl-35', 'Cl-37', 'Ar-36', 'Ar-38', 'Ar-40', 'K-39', 'K-40', 'K-41', 'Ca-40', 'Ca-42', 'Ca-43', 'Ca-44', 'Ca-46', 'Ca-48', 'Sc-45', 'Ti-46', 'Ti-47', 'Ti-48', 'Ti-49', 'Ti-50', 'V-50', 'V-51', 'Cr-50', 'Cr-52', 'Cr-53', 'Cr-54', 'Mn-55', 'Fe-54', 'Fe-56', 'Fe-57', 'Fe-58', 'Co-59', 'Ni-58', 'Ni-60', 'Ni-61', 'Ni-62', 'Ni-64', 'Cu-63', 'Cu-65', 'Zn-64', 'Zn-66', 'Zn-67', 'Zn-68', 'Zn-70', 'Ga-69', 'Ga-71', 'Ge-70', 'Ge-72', 'Ge-73', 'Ge-74', 'Ge-76', 'As-75', 'Se-74', 'Se-76', 'Se-77', 'Se-78', 'Se-80', 'Se-82', 'Br-79', 'Br-81', 'Kr-78', 'Kr-80', 'Kr-82', 'Kr-83', 'Kr-84', 'Kr-86', 'Rb-85', 'Rb-87', 'Sr-84', 'Sr-86', 'Sr-87', 'Sr-88', 'Y-89', 'Zr-90', 'Zr-91', 'Zr-92', 'Zr-

In [ ]:
# As an example, here is the mass of He-4 in the circumgalactic medium at the 3rd timestep
i_step = 4
i_isotope = o.inner.history.isotopes.index('He-4')
print(o.ymgal_outer[i_step][i_isotope],'Msun')

2785282647.925115 Msun


### Task 1.a) Plotting the time evolution of masses
In the cell below, create arrays for plotting the following quantities as a function of the age of the galaxy:
* The total mass of gas in the circumgalactic medium (outside the galaxy)
* The total mass of gas inside galaxy
* The total mass of stars
    * **Warning!** You will need to account the mass ejected by stars.
    * m_star =  total_mass_of_stars_formed - total_mass_ejected_by_stars

Then, in the next cell, check that you filled the arrays correctly. You need to make sure the total mass is conserved (because now the mass loading is 0, we should not lose any mass from the simulation). The three components of masses listed above are the all forms of masses within the simulation.
The total amount of matter should be the same (within 10% error) from one timestep to another.

Then plot the arrays as a function of time.

In [ ]:
# Declaration of the arrays
m_gas_circumgalactic = np.zeros(o.inner.nb_timesteps) # [M_sun]
m_gas_galaxy = np.zeros(o.inner.nb_timesteps) # [M_sun]
m_star = np.zeros(o.inner.nb_timesteps) # [M_sun]

###
### Fill the arrays ###
###

#for i_step in range(o.inner.nb_timesteps):
    # ...

In [ ]:
###
### Fill this cell: mass conservation ###
###

In [ ]:
plt.plot(t_sim, m_gas_circumgalactic)
plt.plot(t_sim, m_gas_galaxy)
plt.plot(t_sim, m_star)

###
### Add legend and axis labels
###

### Task 1.b) Experiment with the input parameters
Now that you have a plotting routine that shows you the evolution of masses, you can start discovering how the different input parameters will change the results in time. Write down a few sentences about your conclusions in each case.

I) Re-run the model with star formation switched on.
* Explore different "sfe" values between 0.001 and 0.05 (re-run OMEGA+, see above).
* Understand the role of star formation, and try to predict what will happen when you change "sfe".

*Put your answer here:*

II) Re-run the model with star formation and galactic outflows switched on.
* Set "sfe = 0.01"
* Explore different "mass_loading" values between 1.0 and 5.0.
* Understand the role of galactic outflows, and try to predict what will happen when you change "mass_loading".

*Put your answer here:*

III) Try to find a set of parameters that respect the following criteria:
* About $6 \cdot 10^{10}$ M$_\odot$ of stars formed.
* The mass of the circumgalactic medium must be larger than $6 \cdot 10^{10}$ M$_\odot$.

*Put your answer here:*

# 2. Calibration of the Milky Way Model

Here you will calibrate your simple Milky Way model against observational constraints. These observed properties of the Milky Way are the following:
* **Total stellar mass formed**: this is the mass of the stars that we currently see in the Milky Way. We want to reproduce this by the end of the simulation.
* **Final star formation rate**: this is the star formation rate that we see today in the Milky Way.
* **Composition at the birth of the Sun**: We know the composition of the Sun well. The Sun formed from the gas 4.6 Gyr ago, we therefore can use the Sun as a guideline for the composition of the gas at that epoch.

You only need to work with 2 parameters (star formation and galactic inflows). You need to vary the parameters until you respect all 3 observational constraints. You will then use this calibrated model to investigate the origin of the elements in our galaxy.

**Useful information to help you calibrate your model**
* The star formation efficiency (sfe) mostly affects [Fe/H].
* The amount of galactic inflows (in_mag) mostly affect the total stellar mass formed.

### Task 2): Calibrate the Milky Way model
Find the best set of parameters that reproduce the observables, using the cells below. Briefly explain what effects the different input parameters have on the results.

*Explanation:*

#### Run your model

In [ ]:
# Star formation efficiency.
# This is controling how fast the gas inside the galaxy is converted into stars.
# You can vary this parameter between 0.005 and 0.05
sfe = 0.005

# Magnitude (strength) of the galactic inflow rate.
# A higher "in_mag" value means more inflows.
# You can vary this parameter between 0.5 and 2.0
in_mag = 0.5

# !!Do not modify the following line!!
# Run OMEGA+ with the first set of parameters
o_mw = omega_plus.omega_plus(sfe=sfe, t_star=1.0e8, special_timesteps=90, \
                 exp_infall=[[in_mag*40, 0.0, 0.8e9], [in_mag*5, 1.0e9, 7.0e9]], \
                 table='yield_tables/agb_and_massive_stars_K10_K06_0.5HNe.txt', \
                 mass_loading=0.5, imf_yields_range=[1,40])

#### Constraint #1 - Total stellar mass formed
Your model should form in total in between $4.5\times10^{10}$ and $5.0\times10^{10}$ M$_\odot$ of stars.

In [ ]:
# Print the total stellar mass formed
print("Total stellar mass formed:",'%.2e'%sum(o_mw.inner.history.m_locked),'M_sun')

#### Constraint #2 - Final star formation rate
The final star formation rate (SFR) of your model should be within the cyan/blue vertical band (in between 0.6 and 3.0 M$_\odot$/yr).

In [ ]:
# Plot the evolution of the star formation rate (SFR)
o_mw.inner.plot_star_formation_rate(color='r', marker='o', label='My model')
plt.xlabel('Galactic age [yr]')

# Plot the observational constraint (cyan color)
plt.plot([13e9,13e9], [0.65,3.0], linewidth=6, color='c', alpha=0.5)

#### Constraint #3 - Solar value [Fe/H=0] by the time the Sun forms
Your prediction needs to go through the intersection between the horizontal and vertical dotted lines, that marks the composition of the Sun.

This is using the following spectroscopic notation,

$$\mathrm{[Fe/H]}=\log(n_\mathrm{Fe}/n_\mathrm{H})-\log(n_\mathrm{Fe}/n_\mathrm{H})_\odot$$

where n_X is the number density of the element X. [X/Y] ratios, where X and Y can be any elements, are expressed in log-space and are normalized to the solar composition.

In [ ]:
# Plot the evolution of [Fe/H], the iron abundance of the gas inside the galaxy.
o_mw.inner.plot_spectro(color='r', marker='o', label='My model')
plt.xlabel('Galactic age [yr]')
plt.ylim(-2,1)

# Plot the solar value (black dotted lines)
t_Sun = 13.0e9 - 4.6e9
plt.plot([t_Sun,t_Sun], [-2,1], ':k')
plt.plot([0,13e9], [0,0], ':k')

# 3. Contribution of Different Enrichment Sources

Now that you have a first calibration of your Milky Way model, you can start looking at the evolution of the elements (and also isotopes if you want). This plot will be a useful test for later tasks.

### Select your element (or isotope)

In [ ]:
# Target element (or isotope)
# Element nomenclature: First letter in capital. e.g., C, N, O, Mg, ..
# Isotope nomenclature: O-16, Fe-56, ..
specie = 'N'

# Verify that the target specie is available
if specie in o_mw.inner.history.elements or specie in o.inner.history.isotopes:
    print('The element/isotope is available.')
else:
    print('Error - The element/isotope IS NOT available, please select a new one.')

### Plot the evolution of the selected element
This will show the mass of the element in form of gas **inside the galaxy**. It will also tell you which type of astronomical events produce the most of this element (or isotope) in the Milky Way galaxy.

In [ ]:
o_mw.inner.plot_mass(specie=specie, source='massive', label='Massive stars')
o_mw.inner.plot_mass(specie=specie, source='agb', label='Low-mass stars')
o_mw.inner.plot_mass(specie=specie, source='sn1a', label='Type Ia supernovae')
o_mw.inner.plot_mass(specie=specie, label='Total mass')
plt.xlabel('Galactic age [yr]')
plt.ylabel('Mass of '+specie)
#plt.xscale('linear')
#plt.ylim(1e3,6e7)

### Task 3): Experiment with the evolution of elements

Try out different elements and see what contributions they have from different sources over time. Does it fit well into your knowledge about GCE?

Tip: the second periodic table in the lecture may be useful.

# 4. Chemical Evolution Trends of Abundances

Here you are comparing your Milky Way model against observations of stellar abundances, using the STELLAB library.

You can investigate the evolution of elemental ratios as a function of time (or rather as a function of [Fe/H]) throughout the history of the Milky Way. Stars formed at different epochs, with different [Fe/H], and preserve the current composition of the gas at that time.

You see an example of such a plot below, showing [Ca/Fe] as a function of [Fe/H].

In [ ]:
# Load the stellar abundance data
s = stellab.stellab()

# Create a list of reference observational papers.
obs = ['stellab_data/milky_way_data/Cohen_et_al_2013_stellab',\
       'stellab_data/milky_way_data/Jacobson_et_al_2015_stellab',\
       'stellab_data/milky_way_data/Venn_et_al_2004_stellab',\
       'stellab_data/milky_way_data/Bensby_et_al_2014_stellab',\
       'stellab_data/milky_way_data/Nissen_et_al_2014_stellab',\
       'stellab_data/milky_way_data/Battistini_Bensby_2015_stellab',\
       'stellab_data/milky_way_data/Battistini_Bensby_2016_stellab']

In [ ]:
# Select your abundance ratios.
xaxis = '[Fe/H]'
yaxis = '[Ca/Fe]'

# Plot data using your selection of data points
s.plot_spectro(xaxis=xaxis, yaxis=yaxis, norm='Asplund_et_al_2009', obs=obs)

# Extract the numerical prediction of OMEGA+ using the "return_x_y" argument.
xy_o = o_mw.inner.plot_spectro(xaxis=xaxis, yaxis=yaxis, return_x_y=True, solar_norm='Asplund_et_al_2009')
plt.plot(xy_o[0], xy_o[1], color='w', linewidth=3.0)
plt.plot(xy_o[0], xy_o[1], color='k', linewidth=1.5, label='Your model')

# Set the range of the x and y axis
plt.legend(loc='center left', bbox_to_anchor=(1.01, 0.5), markerscale=0.8, fontsize=14)
plt.xlim(-4.5,0.75)
plt.ylim(-1.0,1.)

### Task 4): Understanding abundance behaviours

4.a) Understand why **[Mg/Fe]-vs-[Fe/H]** is bending down and decreasing at [Fe/H] > -1.
* The analysis plots shown in the previous sections will be useful.


In [ ]:
# Make your plots and analysis here
# Also write you answers

4.b) Understand why **[C/O]-vs-[Fe/H]** is increasing at [Fe/H] > -2.
* Don't worry too much if the fit is not perfect.

In [ ]:
# Make your plots and analysis here
# Also write you answers

4.c) Understand why **[Ca/Si]-vs-[Fe/H]** is increasing at [Fe/H] > -2.
* Don't worry too much if the fit is not perfect.

In [ ]:
# Make your plots and analysis here
# Also write you answers

4.d) Can you explain why **[Ti/Fe]-vs-[Fe/H]** does not fit?
* By only looking at the observed data, can you guess in which stars Ti should be produced?

In [ ]:
# Make your plots and analysis here
# Also write you answers

# 5. The Origin of "Light" Elements in our Solar System

Besides stellar abundances data, the abundance distribution of the Sun can be used to understand the origin of the elements. The solar composition represents only one timeframe in the history of the Milky Way, but the advantage is that there is data for all the stable elements and isotopes. Here you will compare your chemical evolution predictions with the solar composition.

You will import the solar abundances as measured spectroscopically. The read-in of the data is handled in a separate script. You should run the script with the cell below. **Important: read the comments in the cell, the execution depends on what machine you run the notebook.**

The script will provide you with the array 'solar_Z' and 'solar_ab'. 'solar_Z' contains the atomic number Z of all elements measured in the Sun, while 'solar_ab' lists the corresponding abundances (in mass fraction).

The script also contains a function that helps you to extract the different sources for an element from your simulation. You can call it with 'get_individual_sources', an example will be shown below.

In [ ]:
# Load the functions for extracting abundances distributions.
# You will need to load a script for this.

# If you run the notebook in Colab, upload the script and Lodders_et_al_2009.txt
# to your Drive, into the Colab Notebooks folder, then execute the cell like this.
from google.colab import drive
drive.mount('/content/gdrive')
!cp gdrive/My\ Drive/Colab\ Notebooks/script_solar_ab.py .
!cp gdrive/My\ Drive/Colab\ Notebooks/Lodders_et_al_2009.txt .

# If you run the notebook on you machine, you only need the line below.
# BUT: don't forget to place the files script_solar_ab.py and
# Lodders_et_al_2009.txt in the folder in which you are running the notebook!
# Comment the lines above in this cell.
%run script_solar_ab.py

## 5.1 The Role of Type Ia Supernovae

Here you will vary the number of Type Ia Supernovae until you match the solar composition. Keep in mind that Type Ia Supernovae mostly produce iron and other iron-group elements (e.g. Fe, Co, Ni).

### Run the model

In [ ]:
# Number of Type Ia supernova per units of stellar mass formed.
# You can vary this parameters between 1.0e-4 and 1.0e-1
nb_1a_per_m = 1e-1

# Run OMEGA+ with your set of parameters (simple Milky Way model).
o_51 = omega_plus.omega_plus(sfe=sfe, t_star=1.0e8, special_timesteps=120, \
                 exp_infall=[[in_mag*40, 0.0, 0.8e9], [in_mag*5, 1.0e9, 7.0e9]], \
                 table='yield_tables/agb_and_massive_stars_K10_K06_0.5HNe.txt', \
                 mass_loading=0.5, imf_yields_range=[1,40], nb_1a_per_m=nb_1a_per_m)

In [ ]:
###
### Fill the cell ###
###

# Write a script to find the timestep where the Sun should form.
# You need to look in the metallicity array and find the timestep
# index where the metallicity is the closest to the solar value (0.014).
# Don't find it by hand, write a script to find it.
#
# Metallicity array --> o_51.inner.history.metallicity[ timestep index ]

# Timestep index where the Sun should aproximately form.
# i_t_Sun = place your index here

In [ ]:
# Get source contributions.
# This will use the i_t_Sun parameter found in the previous cell.
m_el_all, m_el_agb, m_el_massive, m_el_sn1a, m_el_nsm = \
    get_individual_sources(o_51.inner, i_step_sol=i_t_Sun)

### Plot the abundance distribution
You do not need to change anything in the cell below.

The plot will show you the mass fractions of the different elements in the Sun, as well as in your simulation. You can see the total mass fraction of your simulation as well as the contribution from different sources.

In [ ]:
# Define the figure size
fig = plt.figure(figsize=(8,4.0))
matplotlib.rcParams.update({'font.size': 14.0})

# Plot solar abundance data
plt.plot(solar_Z, solar_ab, color='k', marker='o', linewidth=4, alpha=0.5, label='Solar')

# All sources combined
plt.plot(Z_charge, m_el_all, color='orange', label='All sources', alpha=1.0, linestyle='-', linewidth=2)

# Type Ia supernovae
plt.plot(Z_charge, m_el_sn1a, color='g', label='SNe Ia', alpha=0.8, linestyle='-', marker='s')

# Massive stars (core-collapse supernovae)
plt.plot(Z_charge, m_el_massive, color='b', label='Massive', alpha=0.8, linestyle='-', marker='^')

# Add element annotations
for i in range(0,nb_elements):
    plt.annotate(elements[i], xy=(solar_Z[i],yy[i]), color='k',\
                 fontsize=13, ha='center', va='center')

# Label and axis
plt.legend(fontsize=13, loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('Z (charge number)', fontsize=16)
plt.ylabel('X (mass fraction)', fontsize=16)
plt.xlim(19.5,30.5)
plt.ylim(1e-9,3e-1)
plt.yscale('log')

# Frame tuning
plt.subplots_adjust(top=0.95)
plt.subplots_adjust(right=0.75)
plt.subplots_adjust(left=0.15)
plt.subplots_adjust(bottom=0.14)

### Task 5): Reproduce the solar iron-peak elements
5.a) Modify the number of Type Ia supernovae in your model until your reproduce the solar composition for **Fe, Co, and Ni**.

5.b) How many Type Ia supernovae you need in total in the Milky Way to have enough Fe, Co, and Ni?
* The number of Type Ia supernovae occuring during each timestep is stored in the following array: **o_51.inner.history.sn1a_numbers**[ timestep index ]

In [ ]:
# Add your code and solution here.

5.c) How well do the other elements fit? What process should be adjusted in the simulation, in your opinion, and why?

*Answer:*

# 6. The Origin of Neutron-Capture Elements in our Solar System

In the following, you will assume that neutron star mergers are the only source of r-process elements in the Milky Way.  This may not be true, as core-collapse supernovae from massive stars could also contribute.

In [ ]:
# Number of neutron star mergers per units of stellar mass formed.
# You can vary this parameters between 1.0e-6 and 1.0e-4
nb_nsm_per_m = 1.0e-6

# Run OMEGA+ with your set of parameters (simple Milky Way model).
r_process_yields = 'yield_tables/r_process_arnould_2007.txt'
o_6 = omega_plus.omega_plus(sfe=sfe, t_star=1.0e8, special_timesteps=120, \
                 exp_infall=[[in_mag*40, 0.0, 0.8e9], [in_mag*5, 1.0e9, 7.0e9]], \
                 mass_loading=0.5, imf_yields_range=[1,40], nb_1a_per_m=nb_1a_per_m, \
                 ns_merger_on=True, nb_nsm_per_m=nb_nsm_per_m, t_nsm_coal=1.0e8, \
                 nsmerger_table=r_process_yields)

# Get source contributions (this needs to be right after running OMEGA+)
m_el_all_6, m_el_agb_6, m_el_massive_6, m_el_sn1a_6, m_el_nsm_6 = \
    get_individual_sources(o_6.inner, i_step_sol=i_t_Sun)

In [ ]:
# Define the figure size
fig = plt.figure(figsize=(8,4.0))
matplotlib.rcParams.update({'font.size': 14.0})

# Plot solar abundance data
plt.plot(solar_Z, solar_ab, color='k', marker='o', linewidth=4, alpha=0.5, label='Solar')

# All sources combined
plt.plot(Z_charge, m_el_all_6, color='orange', label='All sources', alpha=1.0, linestyle='-', linewidth=2)

# AGB stars
plt.plot(Z_charge, m_el_agb_6, color='r', label='AGB', alpha=0.8, linestyle='-', marker='x')

# Neutron star mergers (r-process)
plt.plot(Z_charge, m_el_nsm_6, color='c', label='NS mergers', alpha=0.8, linestyle='-', marker='^')

# Add element annotations
for i in range(0,nb_elements):
    plt.annotate(elements[i], xy=(solar_Z[i],yy[i]), color='k',\
                 fontsize=13, ha='center', va='center')

# Label and axis
plt.legend(fontsize=13, frameon=False)
plt.xlabel('Z (charge number)', fontsize=16)
plt.ylabel('X (mass fraction)', fontsize=16)
plt.xlim(30.5,82.5)
plt.ylim(1e-11,1e-4)
plt.yscale('log')

# Frame tuning
plt.subplots_adjust(top=0.95)
plt.subplots_adjust(right=0.95)
plt.subplots_adjust(left=0.15)
plt.subplots_adjust(bottom=0.14)

### Task 6): Adjust neutron star mergers
6.a) Modify the number of neutron star merger in your model until your reproduce the solar composition for **Te, I, Xe, Os, Ir, Pt, and Au**.  

6.b) How many neutron star merger you need in total in the Milky Way to have enough gold (Au)?
* The number of neutron star merger occuring during each timestep is stored in the following array: **o_6.inner.history.nsm_numbers**[ timestep index ]

In [ ]:
# Add your code and solution here.

### Final task
Go through the notebook and see in what steps you calibrated your model. Which parameters did you finetune? What constraints did you use to tune them?

# + Isotopic Composition of the Sun

At the time the Sun forms in your model, when you fit the abundance of an element, do you also fit the isotopic distribution for that element?

This is only if you have time..

In [ ]:
# If you are using Google Colab, you need the line below. If not, comment it out.
!cp gdrive/My\ Drive/Colab\ Notebooks/Asplund_et_al_2009_iso.txt .

# If you run it on your machine: don't forget to place the file
# Asplund_et_al_2009_iso.txt into the folder of your notebook!

# Extract the list of isotopes mass fraction for the Sun
iso_A09_name = []
iso_A09_f = []
with open('Asplund_et_al_2009_iso.txt') as ff:
    for line in ff:
        line_split = [str(x) for x in line.split()]
        iso_A09_name.append(line_split[0])
        iso_A09_f.append(float(line_split[1])/100.0)
ff.close()

In [ ]:
# For each isotope in the Sun, this is the mass contribution to the total mass of the associated element.
# For example, "Ca-40 0.96941" means that the Ca-40 isotope represent 96.941% of the mass of the Ca element.
for i_iso in range(len(iso_A09_name)):
    print(iso_A09_name[i_iso], iso_A09_f[i_iso])

### Task 7), optional: isotopic distribution

Pick one element that you want to study. Compare its isotopic abundances to the solar composition. Can you reproduce it with your model? You should create plots similar to the ones above.

In [ ]:
# Add your code and solution here.